In [1]:
import pandas as pd
import numpy as np
import uproot
import matplotlib
from matplotlib import pyplot as plt
from __future__ import division
from tqdm import tqdm

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

normalize = False
hard_cuts = True
cut = 'tight'

In [2]:
in_dir = 'trees/'#box_selected/'

input_bkg = ['trees/MaxDigis_tree_Data.root',]
input_sig = [ #in_dir+'MaxDigis_tree_HTo2LongLivedTo4b_MH-125_MFF-25_CTau-15000mm_TuneCP5_14TeV_pythia8.root',
              #in_dir+'MaxDigis_tree_HTo2LongLivedTo4b_MH-125_MFF-25_CTau-1500mm_TuneCP5_14TeV_pythia8.root',
              #in_dir+'MaxDigis_tree_HTo2LongLivedTo4b_MH-125_MFF-12_CTau-9000mm_TuneCP5_14TeV_pythia8.root',
              #in_dir+'MaxDigis_tree_HTo2LongLivedTo4b_MH-125_MFF-12_CTau-900mm_TuneCP5_14TeV_pythia8.root',
              #in_dir+'MaxDigis_tree_HTo2LongLivedTo4b_MH-125_MFF-50_CTau-30000mm_TuneCP5_14TeV_pythia8.root',
              #in_dir+'MaxDigis_tree_HTo2LongLivedTo4b_MH-125_MFF-50_CTau-3000mm_TuneCP5_14TeV_pythia8.root',
              #in_dir+'MaxDigis_tree_HTo2LongLivedTo4b_MH-250_MFF-120_CTau-10000mm_TuneCP5_14TeV_pythia8.root',
              #in_dir+'MaxDigis_tree_HTo2LongLivedTo4b_MH-250_MFF-120_CTau-1000mm_TuneCP5_14TeV_pythia8.root',
              #in_dir+'MaxDigis_tree_HTo2LongLivedTo4b_MH-250_MFF-120_CTau-500mm_TuneCP5_14TeV_pythia8.root',
              #in_dir+'MaxDigis_tree_HTo2LongLivedTo4b_MH-250_MFF-60_CTau-10000mm_TuneCP5_14TeV_pythia8.root',
              #in_dir+'MaxDigis_tree_HTo2LongLivedTo4b_MH-250_MFF-60_CTau-500mm_TuneCP5_14TeV_pythia8.root',
              #in_dir+'MaxDigis_tree_HTo2LongLivedTo4b_MH-350_MFF-160_CTau-10000mm_TuneCP5_14TeV_pythia8.root',
              #in_dir+'MaxDigis_tree_HTo2LongLivedTo4b_MH-350_MFF-160_CTau-1000mm_TuneCP5_14TeV_pythia8.root',
              #in_dir+'MaxDigis_tree_HTo2LongLivedTo4b_MH-350_MFF-160_CTau-500mm_TuneCP5_14TeV_pythia8.root',
              #in_dir+'MaxDigis_tree_HTo2LongLivedTo4b_MH-350_MFF-80_CTau-10000mm_TuneCP5_14TeV_pythia8.root',
              #in_dir+'MaxDigis_tree_HTo2LongLivedTo4b_MH-350_MFF-80_CTau-1000mm_TuneCP5_14TeV_pythia8.root',
              #in_dir+'MaxDigis_tree_HTo2LongLivedTo4b_MH-350_MFF-80_CTau-500mm_TuneCP5_14TeV_pythia8.root',
              #in_dir+'MaxDigis_tree_HTo2LongLivedTo4b_MH-1000_MFF-450_CTau-100000mm_TuneCP5_14TeV_pythia8.root',
              #in_dir+'MaxDigis_tree_HTo2LongLivedTo4b_MH-1000_MFF-450_CTau-10000mm_TuneCP5_14TeV_pythia8.root',
              #in_dir+'MaxDigis_tree_HTo2LongLivedTo4q_MH_125_MFF_1_CTau_10000mm_TuneCP5_14TeV_pythia.root',
              in_dir+'MaxDigis_tree_HTo2LongLivedTo4q_MH_125_MFF_1_CTau_5000mm_TuneCP5_14TeV_pythia.root',
             ]

comp_treename = 'comparator'
wire_treename = 'wire'

comp_var = [ 'Ev_max_nComp_ME11','Ev_max_nComp_ME12','Ev_max_nComp_ME13','Ev_max_nComp_ME21','Ev_max_nComp_ME22',
             'Ev_max_nComp_ME31','Ev_max_nComp_ME32','Ev_max_nComp_ME41','Ev_max_nComp_ME42']
wire_var = [ 'Ev_max_nWire_ME11','Ev_max_nWire_ME12','Ev_max_nWire_ME13','Ev_max_nWire_ME21','Ev_max_nWire_ME22',
             'Ev_max_nWire_ME31','Ev_max_nWire_ME32','Ev_max_nWire_ME41','Ev_max_nWire_ME42']

comp_vars = list(set(comp_var))
wire_vars = list(set(wire_var))

arrs_comp_sig = [None]*(len(input_sig)+1)
arrs_wire_sig = [None]*(len(input_sig)+1)

print "Opening input files"

print "Opening file: ", input_sig[0]
arrs_comp_sig[0] = uproot.open(input_sig[0])[comp_treename]
arrs_wire_sig[0] = uproot.open(input_sig[0])[wire_treename]
comp_sig = arrs_comp_sig[0].pandas.df(comp_vars)
wire_sig = arrs_wire_sig[0].pandas.df(wire_vars)

for i in range(1,len(input_sig)):
    print "Opening file: ", input_sig[i]
    arrs_comp_sig[j] = uproot.open(input_sig[i])[comp_treename]
    arrs_wire_sig[j] = uproot.open(input_sig[i])[wire_treename]
    temp_comp_sig = arrs_comp_sig[i].pandas.df(comp_vars)
    temp_wire_sig = arrs_wire_sig[i].pandas.df(wire_vars)
    if normalize:
        temp_comp_sig = temp_comp_sig.head(130)
        temp_wire_sig = temp_wire_sig.head(130)
    #print "Comp num events = %i" % (len(temp_comp_sig.index))
    #print "Wire num events = %i" % (len(temp_wire_sig.index))
    comp_sig = comp_sig.append(temp_comp_sig)
    wire_sig = wire_sig.append(temp_wire_sig)

#print "Opening file: ", input_bkg[0]
arrs_comp_bkg = uproot.open(input_bkg[0])[comp_treename]
arrs_wire_bkg = uproot.open(input_bkg[0])[wire_treename]
comp_bkg = arrs_comp_bkg.pandas.df(comp_vars)
wire_bkg = arrs_wire_bkg.pandas.df(wire_vars)
#print "Number of bkg events: %i" % (len(wire_bkg))
#print "Number of sig events: %i" % (len(wire_sig))

comp_sig_tot = len(comp_sig)
wire_sig_tot = len(wire_sig)

Opening input files
Opening file:  trees/MaxDigis_tree_HTo2LongLivedTo4q_MH_125_MFF_1_CTau_5000mm_TuneCP5_14TeV_pythia.root


In [3]:
#compXX = [ 98, 56, 30, 49, 42, 49, 35, 42, 31] #nominal
#compXX = [99, 73, 31, 53, 46, 50, 36, 49, 36] #tight
compXX = [99, 57, 25, 45, 36, 43, 29, 43, 32] #loose
#wireXX = [104,  99, 32, 133, 83, 130, 74, 127, 88] #nominal
#wireXX = [105, 102, 48,  134, 84,  131, 87,  128, 94] #tight
wireXX = [105, 93, 33, 134, 80, 118, 75, 128, 87] #loose

efficiency_comp_final = len(comp_sig[(comp_sig['Ev_max_nComp_ME11'] > compXX[0]) | (comp_sig['Ev_max_nComp_ME12'] > compXX[1]) | (comp_sig['Ev_max_nComp_ME13'] > compXX[2]) |
                           (comp_sig['Ev_max_nComp_ME21'] > compXX[3]) | (comp_sig['Ev_max_nComp_ME22'] > compXX[4]) |
                           (comp_sig['Ev_max_nComp_ME31'] > compXX[5]) | (comp_sig['Ev_max_nComp_ME32'] > compXX[6]) |
                           (comp_sig['Ev_max_nComp_ME41'] > compXX[7]) | (comp_sig['Ev_max_nComp_ME41'] > compXX[8])])/comp_sig_tot*100
efficiency_wire_final = len(wire_sig[(wire_sig['Ev_max_nWire_ME11'] > wireXX[0]) | (wire_sig['Ev_max_nWire_ME12'] > wireXX[1]) | (wire_sig['Ev_max_nWire_ME13'] > wireXX[2]) |
                           (wire_sig['Ev_max_nWire_ME21'] > wireXX[3]) | (wire_sig['Ev_max_nWire_ME22'] > wireXX[4]) |
                           (wire_sig['Ev_max_nWire_ME31'] > wireXX[5]) | (wire_sig['Ev_max_nWire_ME32'] > wireXX[6]) |
                           (wire_sig['Ev_max_nWire_ME41'] > wireXX[7]) | (wire_sig['Ev_max_nWire_ME41'] > wireXX[8])])/wire_sig_tot*100

print "File: ", input_sig[0]
print "Loose cut"
print "Comparator Efficiency = %f%%" %(efficiency_comp_final)
print "Wire Efficiency = %f%%" %(efficiency_wire_final)

File:  trees/MaxDigis_tree_HTo2LongLivedTo4q_MH_125_MFF_1_CTau_5000mm_TuneCP5_14TeV_pythia.root
Loose cut
Comparator Efficiency = 22.297297%
Wire Efficiency = 12.903226%


In [4]:
compXX = [ 98, 56, 30, 49, 42, 49, 35, 42, 31] #nominal
#compXX = [99, 73, 31, 53, 46, 50, 36, 49, 36] #tight
#compXX = [99, 57, 25, 45, 36, 43, 29, 43, 32] #loose
wireXX = [104,  99, 32, 133, 83, 130, 74, 127, 88] #nominal
#wireXX = [105, 102, 48,  134, 84,  131, 87,  128, 94] #tight
#wireXX = [105, 93, 33, 134, 80, 118, 75, 128, 87] #loose

efficiency_comp_final = len(comp_sig[(comp_sig['Ev_max_nComp_ME11'] > compXX[0]) | (comp_sig['Ev_max_nComp_ME12'] > compXX[1]) | (comp_sig['Ev_max_nComp_ME13'] > compXX[2]) |
                           (comp_sig['Ev_max_nComp_ME21'] > compXX[3]) | (comp_sig['Ev_max_nComp_ME22'] > compXX[4]) |
                           (comp_sig['Ev_max_nComp_ME31'] > compXX[5]) | (comp_sig['Ev_max_nComp_ME32'] > compXX[6]) |
                           (comp_sig['Ev_max_nComp_ME41'] > compXX[7]) | (comp_sig['Ev_max_nComp_ME41'] > compXX[8])])/comp_sig_tot*100
efficiency_wire_final = len(wire_sig[(wire_sig['Ev_max_nWire_ME11'] > wireXX[0]) | (wire_sig['Ev_max_nWire_ME12'] > wireXX[1]) | (wire_sig['Ev_max_nWire_ME13'] > wireXX[2]) |
                           (wire_sig['Ev_max_nWire_ME21'] > wireXX[3]) | (wire_sig['Ev_max_nWire_ME22'] > wireXX[4]) |
                           (wire_sig['Ev_max_nWire_ME31'] > wireXX[5]) | (wire_sig['Ev_max_nWire_ME32'] > wireXX[6]) |
                           (wire_sig['Ev_max_nWire_ME41'] > wireXX[7]) | (wire_sig['Ev_max_nWire_ME41'] > wireXX[8])])/wire_sig_tot*100

print "File: ", input_sig[0]
print "Nominal cut"
print "Comparator Efficiency = %f%%" %(efficiency_comp_final)
print "Wire Efficiency = %f%%" %(efficiency_wire_final)

File:  trees/MaxDigis_tree_HTo2LongLivedTo4q_MH_125_MFF_1_CTau_5000mm_TuneCP5_14TeV_pythia.root
Nominal cut
Comparator Efficiency = 21.621622%
Wire Efficiency = 10.967742%


In [5]:
#compXX = [ 98, 56, 30, 49, 42, 49, 35, 42, 31] #nominal
compXX = [99, 73, 31, 53, 46, 50, 36, 49, 36] #tight
#compXX = [99, 57, 25, 45, 36, 43, 29, 43, 32] #loose
#wireXX = [104,  99, 32, 133, 83, 130, 74, 127, 88] #nominal
wireXX = [105, 102, 48,  134, 84,  131, 87,  128, 94] #tight
#wireXX = [105, 93, 33, 134, 80, 118, 75, 128, 87] #loose

efficiency_comp_final = len(comp_sig[(comp_sig['Ev_max_nComp_ME11'] > compXX[0]) | (comp_sig['Ev_max_nComp_ME12'] > compXX[1]) | (comp_sig['Ev_max_nComp_ME13'] > compXX[2]) |
                           (comp_sig['Ev_max_nComp_ME21'] > compXX[3]) | (comp_sig['Ev_max_nComp_ME22'] > compXX[4]) |
                           (comp_sig['Ev_max_nComp_ME31'] > compXX[5]) | (comp_sig['Ev_max_nComp_ME32'] > compXX[6]) |
                           (comp_sig['Ev_max_nComp_ME41'] > compXX[7]) | (comp_sig['Ev_max_nComp_ME41'] > compXX[8])])/comp_sig_tot*100
efficiency_wire_final = len(wire_sig[(wire_sig['Ev_max_nWire_ME11'] > wireXX[0]) | (wire_sig['Ev_max_nWire_ME12'] > wireXX[1]) | (wire_sig['Ev_max_nWire_ME13'] > wireXX[2]) |
                           (wire_sig['Ev_max_nWire_ME21'] > wireXX[3]) | (wire_sig['Ev_max_nWire_ME22'] > wireXX[4]) |
                           (wire_sig['Ev_max_nWire_ME31'] > wireXX[5]) | (wire_sig['Ev_max_nWire_ME32'] > wireXX[6]) |
                           (wire_sig['Ev_max_nWire_ME41'] > wireXX[7]) | (wire_sig['Ev_max_nWire_ME41'] > wireXX[8])])/wire_sig_tot*100

print "File: ", input_sig[0]
print "Tight cut"
print "Comparator Efficiency = %f%%" %(efficiency_comp_final)
print "Wire Efficiency = %f%%" %(efficiency_wire_final)

File:  trees/MaxDigis_tree_HTo2LongLivedTo4q_MH_125_MFF_1_CTau_5000mm_TuneCP5_14TeV_pythia.root
Tight cut
Comparator Efficiency = 19.594595%
Wire Efficiency = 9.032258%
